In [38]:
import nltk
import numpy as np
import operator as op
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import io
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [39]:
#chargement des donnes 
liste_id = []
liste_label = []
liste_message = []
file_train = 'data/train.xml'
file_dev   = 'data/dev.xml'
file_test  = 'data/test.xml' 

In [40]:
def read_xml_to_dataframe(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        movie = comment.find('movie').text
        review_id = comment.find('review_id').text
        name = comment.find('name').text
        user_id = comment.find('user_id').text
        note = comment.find('note').text.replace(',', '.')
        commentaire = comment.find('commentaire').text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append({
            'movie': movie,
            'review_id': review_id,
            'name': name,
            'user_id': user_id,
            'note': note,
            'commentaire': commentaire
        })

    # Création d'un DataFrame 
    df = pd.DataFrame(data)

    return df

In [41]:
#fonction pour le fichier de test 
def read_xml_to_dataframeTest(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        # Initialisation d'un dictionnaire pour chaque commentaire
        comment_data = {}

        # Extraction de chaque élément si disponible et ajout au dictionnaire
        movie = comment.find('movie')
        if movie is not None:
            comment_data['movie'] = movie.text

        review_id = comment.find('review_id')
        if review_id is not None:
            comment_data['review_id'] = review_id.text

        name = comment.find('name')
        if name is not None:
            comment_data['name'] = name.text

        user_id = comment.find('user_id')
        if user_id is not None:
            comment_data['user_id'] = user_id.text

        # Note is not present in the given data, but if it's sometimes included,
        # you can check for it like this:
        note = comment.find('note')
        if note is not None:
            comment_data['note'] = note.text.replace(',', '.')

        commentaire = comment.find('commentaire')
        if commentaire is not None:
            comment_data['commentaire'] = commentaire.text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append(comment_data)

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df

# Récupération des données dans des dataFrame

In [42]:
#création DataFrame
data_dev   = read_xml_to_dataframe(file_dev)
data_train = read_xml_to_dataframe(file_train)
data_test  = read_xml_to_dataframeTest(file_test)

In [43]:
############"creation de lexique###############
lemmatizer = WordNetLemmatizer()

def lexique_(message):
    lexique =[]
    lexique_unique = []
    for m in range(len(message)):
        liste = nltk.word_tokenize(message[m],language="french")   
        for  i in liste:
            lexique.append(lemmatizer.lemmatize(i))
        #####lexique unique###
    for i in range(len(lexique)):
        if lexique[i] not in lexique_unique :
            lexique_unique.append(lexique[i])
    return lexique_unique

In [45]:
####################Table de H################
def table_h(lexique_unique):
    h = {}
    i = 0
    for mot in lexique_unique:
       h[mot] = i
       i+=1
    return h

In [46]:
###################Matrice#######################
def matrice(message,lexique_unique):
    h = table_h(lexique_unique)
    nombre_lignes = message.shape[0]
    nombre_colones = len(h)
    m = np.zeros((nombre_lignes,nombre_colones))
    
    for i in range(nombre_lignes):
       #print("i",i)
        for mot in lexique_unique:
            j = h[mot]
            #nombre d'occurence de mot dans message [i]
            mot = mot+" "
            nombre_occurence =  message[i].count(mot)
            m[i,j] =  nombre_occurence
              
    print("fin de la foncrion\n") 
    return m

In [47]:
##################SVM############################
def convertir_svm(matrice, etiquettes):
    # Initialisation de la chaîne de caractères qui contiendra le fichier SVM
    svm = ""

    # Parcourt de la matrice de données 
    for i in range(matrice.shape[0]):
        # Ajout de l'étiquette de la classe en début de ligne
        svm += str(etiquettes[i]) + " "
        # Parcourt de chaque colonne de la ligne courante
        for j in range(matrice.shape[1]):
            # Si la valeur de la caractéristique est non nulle
            if matrice[i, j] != 0:
                svm += f"{j+1}:{matrice[i, j]} "
        # Ajout du retour à la ligne à la fin de la ligne courante
        svm += "\n"
    
    return svm

In [48]:
################################################
def file(nom,contenu):
    
    # Ouvrez le fichier en mode écriture
    with io.open(nom, "w", encoding="utf-8") as f:
        # Écrivez vos lignes de string dans le fichier
        f.writelines(contenu)

In [49]:
#récupération des commentaire
message_test = np.array(data_test['commentaire'])
message_dev = np.array(data_dev['commentaire'])
message_train = np.array(data_train['commentaire'])

In [50]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')

def lexique_(messages):
    lemmatizer = WordNetLemmatizer()
    lexique = set()

    for message in messages:
        if message is not None:  # Vérifiez que le message n'est pas None
            tokens = word_tokenize(message, language="french")
            for token in tokens:
                lexique.add(lemmatizer.lemmatize(token))

    return list(lexique)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\etudiant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [51]:
#creation de lexique
lexique_unique = lexique_(message_train)

In [53]:
lexique_unique

['traduisit',
 'Pic',
 'mono-parentales',
 "s'accomplit",
 'incontestable…',
 'percevérance',
 "L'aversion",
 'Moriarty-Cassady',
 'moyen.Selon',
 'pâlira',
 'Ensuite',
 'quelques',
 "l'arrangera",
 'France.Sincèrement',
 'acupuncteur',
 'klapishiens',
 'NewYork',
 'hyper-saturées',
 'aneaux',
 'dévaliser',
 'Latinos',
 'winstherpoon',
 'nègociateur',
 'cornichonnerie',
 'formèrent',
 'Matriochka',
 'fatigués',
 'Assassinats',
 'poétisée',
 'Jaques-Yves',
 'trouvables',
 'Dawe',
 'Stapple',
 'machinéen',
 'aménées',
 'ni-plus',
 'reboots/remakes/spin-offs',
 'tortueuse',
 'Schwarziennes',
 'sur-dosées',
 'grimait',
 'Évitable',
 'blockbuster^^',
 '//www.cineserie.com/dossiers/gros-plan/gros-plan-john-wick-retour-sur-une-des-meilleures-sagas-daction-contemporaines-2549560/',
 'rembourrage',
 "L'escalavage",
 'offrirai',
 'effectués',
 'blondeur',
 'emennant',
 'zappette',
 'protectrices',
 'menace…',
 'bisexuelle',
 'satiriser',
 'polygamie',
 'Retro',
 'blafarde.Pas',
 'Alexsandr',
 'm

In [55]:
from scipy.sparse import lil_matrix
import numpy as np

def matrice(message, lexique_unique):
    h = table_h(lexique_unique)
    nombre_lignes = message.shape[0]
    nombre_colones = len(h)
    
    # Utilisation d'une matrice creuse
    m = lil_matrix((nombre_lignes, nombre_colones), dtype=np.float32)

    for i in range(nombre_lignes):
        for mot in lexique_unique:
            if mot in message[i]:  # Seulement traiter si le mot est dans le message
                j = h[mot]
                mot += " "
                nombre_occurence = message[i].count(mot)
                m[i, j] = nombre_occurence

    print("fin de la foncrion\n") 
    return m.tocsr()  # Convertir en format CSR pour une meilleure efficacité


In [ ]:
#creaton de la matrice
#donnees_train= matrice(message_train,lexique_unique)
#donnees_test= matrice(message_test,lexique_unique)
donnees_dev= matrice(message_dev,lexique_unique)

In [ ]:
donnees_train= matrice(message_train,lexique_unique)
donnees_test= matrice(message_test,lexique_unique)

In [ ]:
#format accepter par svm
svm_train = convertir_svm(donnees_train , label_train )
svm_test = convertir_svm(donnees_test , label_test )
svm_dev = convertir_svm(donnees_dev , label_dev )

#ecrire dans le fichier
file("train.svm",svm_train)
file("test.svm",svm_test)
file("dev.svm",svm_dev)